# Data preparation

## Purpose
This notebook prepares the data to be consumed by the next step ([3_modeling_suto](3_modeling_suto.ipynb)).

## Methodology

## WIP - improvements

## Results

## Suggested next steps


# Setup

## Library import
We import all the required Python libraries

In [1]:
%matplotlib inline

import os
import pickle
from typing import List

# Data manipulation
import pandas as pd
import numpy as np

# Visualizations
import matplotlib as plt
from pandas_profiling import ProfileReport
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn2pmml import sklearn2pmml
from sklearn2pmml.pipeline import PMMLPipeline
from pypmml import Model

os.chdir('../')
from src.utils.data_describe import breve_descricao, serie_nulos, cardinalidade
os.chdir('./notebooks/')

# Options for pandas
# pd.options.display.max_columns = None
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# pd.options.display.max_rows = 120

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
    
%autoreload 2

## Parameter definition
We set all relevant parameters for our notebook. By convention, parameters are uppercase, while all the 
other variables follow Python's guidelines.

In [2]:
RAW_FOLDER = '../data/raw/'
INTERIM_FOLDER = '../data/interim/'
REPORTS_FOLDER = '../reports/'
MODELS_FOLDER = '../models/'
RANDOM_STATE = 42


# Data import
We retrieve all the required data for the analysis.

In [3]:
df = pd.read_csv(RAW_FOLDER + 'train.csv', index_col=0)
df_evaluation = df.copy() 
df_evaluation.shape

(1460, 80)

# Data processing

## 1st experiment: the baseline
Here we will:

1. Drop the attributes with more than 80% nulls; and
1. Drop all non_numerical features.

### List of attributes to drop

In [4]:
lst_columns_null = serie_nulos(df_evaluation, corte=0.5).index.tolist()

lst_columns_null

4 atributos/features/campos possuem mais de 0.5 de valores nulos.


['PoolQC', 'MiscFeature', 'Alley', 'Fence']

### List of area attributes

In [5]:
lst_area = [x for x in df_evaluation.columns if ('area' in x.lower()) or ('sf' in x.lower())]

print(f"""There is/are {len(lst_area)} area attributes:
{lst_area}""")

There is/are 14 area attributes:
['LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'PoolArea']


### List of time attributes

With these features, I will create new time features that express the difference between the YrSold and the each other time feature. For example:

- 'YrSold' - 'YearBuilt' = 'YrSold_YearBuilt' -> express the building's age.

The difference will not be calculated between 'YrSold' and 'MoSold' because there's no meaning in doing this. 'MoSold' can be used later to observe if there's a seasonality within a year.

In [6]:
lst_time = [x for x in df_evaluation.columns if ('yr' in x.lower()) or ('year' in x.lower())]
# After reading the data description, I realized that 'MoSold' is a time attribute too.

print(f"""There is/are {len(lst_time)} time attributes:
{lst_time}""")

lst_time.remove('YrSold')

There is/are 4 time attributes:
['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold']


### List of float attributes

In [7]:
lst_float = [
    x for x in df_evaluation.select_dtypes(include='float64').columns.tolist() if (x not in lst_area) and (x not in lst_time)
]

print(f"""There is/are {len(lst_float)} float attributes:
{lst_float}""")

There is/are 1 float attributes:
['LotFrontage']


In [8]:
def create_time_dif_columns(df: pd.DataFrame, lst: List) -> pd.DataFrame:
    df_temp = df.copy()

    for attribute in lst:
        df_temp[attribute + '_YrSold'] = df_temp['YrSold'] - df_temp[attribute]
        df_temp.drop(columns=attribute, inplace=True)
    
    return df_temp


### Data output

In [9]:
df_output = df.copy()
df_output.drop(columns=lst_columns_null, inplace=True)

lst_final_columns = lst_area.copy()
lst_final_columns.extend(lst_time)
lst_final_columns.append('YrSold')
lst_final_columns.append('SalePrice')

df_output = df_output[lst_final_columns]
df_output = create_time_dif_columns(df=df_output, lst=lst_time)

df_output = df_output[[
    'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
    '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF',
    'OpenPorchSF', 'PoolArea', 'YrSold', 'YearBuilt_YrSold', 'YearRemodAdd_YrSold',
    'GarageYrBlt_YrSold', 'SalePrice'
]]

df_output.head()

,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,PoolArea,YrSold,YearBuilt_YrSold,YearRemodAdd_YrSold,GarageYrBlt_YrSold,SalePrice
Id,,,,,,,,,,,,,,,,,,,
1,8450,196.0,706,0,150,856,856,854,0,1710,548,0,61,0,2008,5,5,5.0,208500
2,9600,0.0,978,0,284,1262,1262,0,0,1262,460,298,0,0,2007,31,31,31.0,181500
3,11250,162.0,486,0,434,920,920,866,0,1786,608,0,42,0,2008,7,6,7.0,223500
4,9550,0.0,216,0,540,756,961,756,0,1717,642,0,35,0,2006,91,36,8.0,140000
5,14260,350.0,655,0,490,1145,1145,1053,0,2198,836,192,84,0,2008,8,8,8.0,250000


## Presence of null values

The 1st experiment will be a simple linear regression. This method do not allowed null values in the dataset nor in production model. The following steps we'll fill the null values:

In [10]:
serie_nulos(df_output, corte=0)

2 atributos/features/campos possuem mais de 0 de valores nulos.


GarageYrBlt_YrSold    0.055479
MasVnrArea            0.005479
dtype: float64

### Filling attribute: *GarageYrBlt_YrSold*

This attribute could mean that the building has no garage. For this one we'll just fill the null values with a high number (e.g.: 999).

In [11]:
# This dataframe below shows there is NO information about the attributes with 'Garage' in the name.
df.loc[df['GarageYrBlt'].isna(), [x for x in df.columns if 'arage' in x]].sample(5, random_state=RANDOM_STATE)

,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond
Id,,,,,,,
529,NaN,NaN,NaN,0,0,NaN,NaN
40,NaN,NaN,NaN,0,0,NaN,NaN
387,NaN,NaN,NaN,0,0,NaN,NaN
534,NaN,NaN,NaN,0,0,NaN,NaN
288,NaN,NaN,NaN,0,0,NaN,NaN


In [12]:
df_output[['GarageYrBlt_YrSold']] = df_output[['GarageYrBlt_YrSold']].fillna(999)

df_output.loc[
    df_output['GarageYrBlt_YrSold']==999, [x for x in df_output.columns if 'arage' in x]
].sample(5, random_state=RANDOM_STATE)

,GarageArea,GarageYrBlt_YrSold
Id,,
529,0,999.0
40,0,999.0
387,0,999.0
534,0,999.0
288,0,999.0


### Filling attribute: *MasVnrArea*

As showed above, this attribute or factor has only 8 null value entries (0,5% of total). As a first try, it is possible to delete these 8 entries, but we want to build a data prep pipeline to deal with these cases.

Here we are going to try [KNN](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) approach, without the target or response, obviously.

In [13]:
# Creating a copy of the df_output:
df_prep_MasVnrArea = df_output.copy()

# listing the indexes with null values.
lst_nulls_MasVnrArea = df_prep_MasVnrArea[(df_prep_MasVnrArea['MasVnrArea'].isna())].index.tolist()

# listing the indexes WITHOUT null values.
lst_keep_MasVnrArea = [x for x in df_prep_MasVnrArea.index if x not in lst_nulls_MasVnrArea]

df_prep_MasVnrArea[(df_prep_MasVnrArea['MasVnrArea'].isna())]

,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,PoolArea,YrSold,YearBuilt_YrSold,YearRemodAdd_YrSold,GarageYrBlt_YrSold,SalePrice
Id,,,,,,,,,,,,,,,,,,,
235,7851,NaN,625,0,235,860,860,1100,0,1960,440,288,48,0,2010,8,8,8.0,216500
530,32668,NaN,1219,0,816,2035,2515,0,0,2515,484,0,0,0,2007,50,32,32.0,200624
651,8125,NaN,0,0,813,813,822,843,0,1665,562,0,0,0,2008,1,1,1.0,205950
937,10083,NaN,833,0,343,1176,1200,0,0,1200,555,0,41,0,2009,6,6,6.0,184900
974,11639,NaN,0,0,1428,1428,1428,0,0,1428,480,0,120,0,2008,1,0,1.0,182000
978,4274,NaN,1106,0,135,1241,1241,0,0,1241,569,0,116,0,2007,1,0,0.0,199900
1244,13891,NaN,1386,0,690,2076,2076,0,0,2076,850,216,229,0,2006,0,0,0.0,465000
1279,9473,NaN,804,0,324,1128,1128,903,0,2031,577,0,211,0,2008,6,6,6.0,237000


In [14]:
X_train = df_prep_MasVnrArea.drop(columns=['MasVnrArea', 'SalePrice'], index=lst_nulls_MasVnrArea)
X_test = df_prep_MasVnrArea.drop(columns=['MasVnrArea', 'SalePrice'], index=lst_keep_MasVnrArea)
y_train = df_prep_MasVnrArea.loc[lst_keep_MasVnrArea, ['MasVnrArea']]
y_test = df_prep_MasVnrArea.loc[lst_nulls_MasVnrArea, ['MasVnrArea']]

print(f"""train: X - {X_train.shape}, y - {y_train.shape}
test: X - {X_test.shape}, y - {y_test.shape}""")

train: X - (1452, 17), y - (1452, 1)
test: X - (8, 17), y - (8, 1)


#### Trying to use PMML

In [15]:
pipeline = PMMLPipeline([
    ("knn", KNeighborsRegressor(n_neighbors=5, n_jobs=-1))
])

pipeline.fit(X_train, y_train)

sklearn2pmml(pipeline, MODELS_FOLDER + 'data_prep_knn.pmml', with_repr = True)

y_pred_imputer = pipeline.predict(X_test)

y_pred_imputer

# df_output.loc[lst_nulls_MasVnrArea, 'MasVnrArea_pred'] = y_pred_imputer

# df_output.loc[lst_nulls_MasVnrArea, ['MasVnrArea', 'MasVnrArea_pred']]

array([[450. ],
       [ 97.4],
       [  0. ],
       [ 44.4],
       [114.2],
       [260.8],
       [587. ],
       [103.6]])

#### Using Pickle

In [16]:
pipeline = KNeighborsRegressor(n_neighbors=5, n_jobs=-1)

pipeline.fit(X_train, y_train)

pickle.dump(pipeline, open(MODELS_FOLDER + 'data_prep_knn.pkl', 'wb'))

y_pred_imputer = pipeline.predict(X_test)

y_pred_imputer

array([[450. ],
       [ 97.4],
       [  0. ],
       [ 44.4],
       [114.2],
       [260.8],
       [587. ],
       [103.6]])

In [17]:
df_output.loc[lst_nulls_MasVnrArea, 'MasVnrArea'] = y_pred_imputer
              
df_output.loc[lst_nulls_MasVnrArea, :]

,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,PoolArea,YrSold,YearBuilt_YrSold,YearRemodAdd_YrSold,GarageYrBlt_YrSold,SalePrice
Id,,,,,,,,,,,,,,,,,,,
235,7851,450.0,625,0,235,860,860,1100,0,1960,440,288,48,0,2010,8,8,8.0,216500
530,32668,97.4,1219,0,816,2035,2515,0,0,2515,484,0,0,0,2007,50,32,32.0,200624
651,8125,0.0,0,0,813,813,822,843,0,1665,562,0,0,0,2008,1,1,1.0,205950
937,10083,44.4,833,0,343,1176,1200,0,0,1200,555,0,41,0,2009,6,6,6.0,184900
974,11639,114.2,0,0,1428,1428,1428,0,0,1428,480,0,120,0,2008,1,0,1.0,182000
978,4274,260.8,1106,0,135,1241,1241,0,0,1241,569,0,116,0,2007,1,0,0.0,199900
1244,13891,587.0,1386,0,690,2076,2076,0,0,2076,850,216,229,0,2006,0,0,0.0,465000
1279,9473,103.6,804,0,324,1128,1128,903,0,2031,577,0,211,0,2008,6,6,6.0,237000


In [18]:
df_output.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   LotArea              1460 non-null   int64  
 1   MasVnrArea           1460 non-null   float64
 2   BsmtFinSF1           1460 non-null   int64  
 3   BsmtFinSF2           1460 non-null   int64  
 4   BsmtUnfSF            1460 non-null   int64  
 5   TotalBsmtSF          1460 non-null   int64  
 6   1stFlrSF             1460 non-null   int64  
 7   2ndFlrSF             1460 non-null   int64  
 8   LowQualFinSF         1460 non-null   int64  
 9   GrLivArea            1460 non-null   int64  
 10  GarageArea           1460 non-null   int64  
 11  WoodDeckSF           1460 non-null   int64  
 12  OpenPorchSF          1460 non-null   int64  
 13  PoolArea             1460 non-null   int64  
 14  YrSold               1460 non-null   int64  
 15  YearBuilt_YrSold     1460 non-null   i

### Conclusion

As we can see above, there is no null values. Now let's code our dataprep pipeline:

## Dataprep pipeline

In [26]:
def use_data_prep_baseline(df: pd.DataFrame, target_to_impute: str = 'MasVnrArea') -> pd.DataFrame:
    
    if 'SalePrice' in df.columns:
        df_prepared = df.drop(columns='SalePrice').copy()
    else:
        df_prepared = df.copy()
    
    # listing the indexes with null values.
    lst_nulls = df_prepared[(df_prepared[target_to_impute].isna())].index.tolist()

    # listing the indexes WITHOUT null values.
    lst_keep = [x for x in df_prepared.index if x not in lst_nulls]
    
    X = df_prepared.drop(columns=target_to_impute, index=lst_keep).copy()
    
#     Load model
#     TODO: Add the model as a parameter of this function.
#     model = Model.load(MODELS_FOLDER + 'data_prep_knn.pmml')
    model = pickle.load(open(MODELS_FOLDER + 'data_prep_knn.pkl', 'rb'))
    y_pred = model.predict(X)
    print(y_pred.shape)
    
    df_prepared.loc[lst_nulls, 'MasVnrArea'] = y_pred     
    
    return df_prepared

In [27]:
# This cell represents the final pipeline.

df_test = pd.concat([X_test, y_test], axis=1)

df_test[['GarageYrBlt_YrSold']] = df_test[['GarageYrBlt_YrSold']].fillna(999)

df_teste_prep = use_data_prep_baseline(df_prepared)

df_teste_prep.info()

(8, 1)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 235 to 1279
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   LotArea              8 non-null      int64  
 1   BsmtFinSF1           8 non-null      int64  
 2   BsmtFinSF2           8 non-null      int64  
 3   BsmtUnfSF            8 non-null      int64  
 4   TotalBsmtSF          8 non-null      int64  
 5   1stFlrSF             8 non-null      int64  
 6   2ndFlrSF             8 non-null      int64  
 7   LowQualFinSF         8 non-null      int64  
 8   GrLivArea            8 non-null      int64  
 9   GarageArea           8 non-null      int64  
 10  WoodDeckSF           8 non-null      int64  
 11  OpenPorchSF          8 non-null      int64  
 12  PoolArea             8 non-null      int64  
 13  YrSold               8 non-null      int64  
 14  YearBuilt_YrSold     8 non-null      int64  
 15  YearRemodAdd_YrSold  8 non-null 

In [33]:
df_output.to_parquet(INTERIM_FOLDER + 'df_train_interim_01_baseline.pqt')